In [386]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, SmallInteger, Date, MetaData, ForeignKey, insert, delete, update, bindparam, select, CheckConstraint
from sqlalchemy.sql import functions
import pandas as pd
import datetime

### Setting up connection (to already existing DB)

In [387]:
def new_engine(user, password, host, db):
    eng = f"mysql+pymysql://{user}:{password}!@{host}/{db}"
    print('Connection succesfull')
    return create_engine(eng)

In [388]:
engine = new_engine('root', 'My5qlg3c', 'localhost', 'inlämning2v2')

Connection succesfull


### Creating adress and personer tables

In [394]:
metadata_obj = MetaData()

adresses = Table(
    "adress",
    metadata_obj,    
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('ort', String(50)),
    Column('post_nr', String(5)),
    Column('gata', String(50)),
    Column('hus_nr', String(10))   
)

personer = Table(
    "person",
    metadata_obj,
    Column('sista_4', SmallInteger, primary_key=True, autoincrement=False),
    Column('f_namn', String(30)),
    Column('e_namn', String(30)),
    Column('f_dag', Date, primary_key=True),
    Column('kon', String(1)),
    Column('nat', String(3)),
    Column('adress_id', Integer, ForeignKey("adress.id")),
    Column('inflytt_datum', Date),
    Column('registrerings_datum', Date, default = functions.current_date()),    
    CheckConstraint("f_dag < registrerings_datum", name="fdag_i_framtiden"),
    CheckConstraint("inflytt_datum < registrerings_datum", name="flyttdatum_i_framtiden"),
    CheckConstraint("f_dag > registrerings_datum - 1100000", name="over_110_ar")
)

In [395]:
metadata_obj.create_all(engine)

### Insert data function

**NOTE: that the function takes a table object and a list of dictionaries as an argument**

batch error checking?
date checking, only date limit check!
how to make a person unique (without 12 size bigint) - fname, ename, bday, 

In [396]:
def insert_data(table, dicts):

    try:
        with engine.connect() as connection:
            ins = table.insert()
            connection.execute(ins, dicts)
            print('Succesfull insert')
    except Exception as e: 
        print('Error:', e)

In [397]:
adress_list = [
{'ort': 'Varberg', 'post_nr': '43243', 'gata': 'Fredsgatan', 'hus_nr': '14'},
{'ort': 'Varberg', 'post_nr': '43295', 'gata': 'Vildåsvägen', 'hus_nr': '12'},
{'ort': 'Falkenberg', 'post_nr': '31194', 'gata': 'Jonstorp', 'hus_nr': '205'},
{'ort': 'Tvååker', 'post_nr': '43279', 'gata': 'Ekvägen', 'hus_nr': '9'},
{'ort': 'Kvibille', 'post_nr': '30578', 'gata': 'Västergårdsvägen', 'hus_nr': '1'},
{'ort': 'Anderstorp', 'post_nr': '33431', 'gata': 'Åsliden', 'hus_nr': '5'},
{'ort': 'Torekov', 'post_nr': '26976', 'gata': 'Källrevägen', 'hus_nr': '12'},
{'ort': 'Kungsbacka', 'post_nr': '43431', 'gata': 'Odens Väg', 'hus_nr': '10B'},
{'ort': 'Oskarström', 'post_nr': '31392', 'gata': 'Havstenshult', 'hus_nr': '666'},
{ 'ort': 'Varberg', 'post_nr': '43247', 'gata': 'Lindbergsvägen', 'hus_nr': '94'},
{ 'ort': 'Kumla', 'post_nr': '69292', 'gata': 'Norrsätter', 'hus_nr': '205'},
{ 'ort': 'Halmstad', 'post_nr': '30258', 'gata': 'Kornhillsvägen', 'hus_nr': '26'},
{ 'ort': 'Västra Frölunda', 'post_nr': '42166', 'gata': 'Möttviksvägen', 'hus_nr': '5'},
{ 'ort': 'Varberg', 'post_nr': '43245', 'gata': 'Göteborgsvägen', 'hus_nr': '10'},
{ 'ort': 'Varberg', 'post_nr': '43244', 'gata': 'Platsgatan', 'hus_nr': '4'},
{ 'ort': 'Träslövsläge', 'post_nr': '43275', 'gata': 'Fiskehamnsvägen', 'hus_nr': '5'},
{ 'ort': 'Falkenberg', 'post_nr': '31135', 'gata': 'Mäster Nils Väg', 'hus_nr': '2'},
{ 'ort': 'Långås', 'post_nr': '31193', 'gata': 'Norra Långås', 'hus_nr': '228'},
{ 'ort': 'Varberg', 'post_nr': '43232', 'gata': 'Härdgatan', 'hus_nr': '4'},
{ 'ort': 'Tvååker', 'post_nr': '43278', 'gata': 'Gästgivaregatan', 'hus_nr': '23'},
{ 'ort': 'Veddige', 'post_nr': '43267', 'gata': 'Nygatan', 'hus_nr': '7'},
{ 'ort': 'Vessigebro', 'post_nr': '31165', 'gata': 'Klockarebolet Klockaregården', 'hus_nr': '110'},
{ 'ort': 'Grimeton', 'post_nr': '43298', 'gata': 'Tjärby', 'hus_nr': '35'},
{ 'ort': 'Nol', 'post_nr': '44990', 'gata': 'Skinnbo', 'hus_nr': '230'},
{ 'ort': 'Falkenberg', 'post_nr': '31178', 'gata': 'Utsiktsvägen', 'hus_nr': '1'},
{ 'ort': 'Hägersten', 'post_nr': '12629', 'gata': 'Radiusbacken', 'hus_nr': '19'},
{ 'ort': 'Tvååker', 'post_nr': '43279', 'gata': 'Smeakalles Väg', 'hus_nr': '14'},
{ 'ort': 'Helsingborg', 'post_nr': '25225', 'gata': 'Södergatan', 'hus_nr': '26'},
{ 'ort': 'Bromma', 'post_nr': '16755', 'gata': 'Västerled', 'hus_nr': '31'},
{ 'ort': 'Sundbyberg', 'post_nr': '17466', 'gata': 'Mönstringsvägen', 'hus_nr': '7'},
{ 'ort': 'Tierp', 'post_nr': '81545', 'gata': 'Kräftvägen', 'hus_nr': '3'},
{ 'ort': 'Älvsjö', 'post_nr': '12572', 'gata': 'Fageredsgränd', 'hus_nr': '4'},
{ 'ort': 'Valbo', 'post_nr': '81832', 'gata': 'Sågvreten', 'hus_nr': '30'},
{ 'ort': 'Huddinge', 'post_nr': '14143', 'gata': 'Granitvägen', 'hus_nr': '31'},
{ 'ort': 'Kista', 'post_nr': '16479', 'gata': 'Torneågatan', 'hus_nr': '104'},
{ 'ort': 'Hägersten', 'post_nr': '12642', 'gata': 'Tångvägen', 'hus_nr': '2'},
{ 'ort': 'Stockholm', 'post_nr': '11267', 'gata': 'Primusgatan', 'hus_nr': '57'},
{ 'ort': 'Nässjö', 'post_nr': '57134', 'gata': 'Fagerhemsvägen', 'hus_nr': '2B'},
{ 'ort': 'Huddinge', 'post_nr': '14132', 'gata': 'Glömstavägen', 'hus_nr': '124'},
{ 'ort': 'Älvsjö', 'post_nr': '12560', 'gata': 'Bergtallsvägen', 'hus_nr': '22'},
{ 'ort': 'Stockholm', 'post_nr': '11640', 'gata': 'Södermannagatan', 'hus_nr': '45'},
{ 'ort': 'Sala', 'post_nr': '73336', 'gata': 'Rosenbergsgatan', 'hus_nr': '11'},
{ 'ort': 'Linköping', 'post_nr': '58247', 'gata': 'Bjälbogatan', 'hus_nr': '28'},
{ 'ort': 'Surahammar', 'post_nr': '73538', 'gata': 'Orkestervägen', 'hus_nr': '11'},
{ 'ort': 'Saltsjö-Duvnäs', 'post_nr': '13150', 'gata': 'Patrons Väg', 'hus_nr': '11'},
{ 'ort': 'Uppsala', 'post_nr': '75220', 'gata': 'Klockargatan', 'hus_nr': '6A'},
{ 'ort': 'Johanneshov', 'post_nr': '12148', 'gata': 'Tidaholmsplan', 'hus_nr': '11'},
{ 'ort': 'Vendelsö', 'post_nr': '13671', 'gata': 'Vendelsö Skolväg', 'hus_nr': '16'},
{ 'ort': 'Trollhättan', 'post_nr': '46198', 'gata': 'Assarebo employeesMaden', 'hus_nr': '1'},
{ 'ort': 'Sundbyberg', 'post_nr': '17276', 'gata': 'Villavägen', 'hus_nr': '10'},
{ 'ort': 'Kista', 'post_nr': '16455', 'gata': 'Skagafjordsgatan', 'hus_nr': '13'},
{ 'ort': 'Hargshamn', 'post_nr': '74250', 'gata': 'Gimovägen', 'hus_nr': '18'},
{ 'ort': 'Järfälla', 'post_nr': '17676', 'gata': 'Af Ugglas Väg', 'hus_nr': '24'},
{ 'ort': 'Stockholm', 'post_nr': '11849', 'gata': 'Hornsgatan', 'hus_nr': '29E'},
{ 'ort': 'Trångsund', 'post_nr': '14262', 'gata': 'Spelvägen', 'hus_nr': '12'},
{ 'ort': 'Stockholm', 'post_nr': '11441', 'gata': 'Nybrogatan', 'hus_nr': '68A'},
{ 'ort': 'Täby', 'post_nr': '18777', 'gata': 'Friggavägen', 'hus_nr': '37'},
{ 'ort': 'Uppsala', 'post_nr': '75427', 'gata': 'Von Bahrs Väg', 'hus_nr': '42'}
]

In [398]:
insert_data(adresses, adress_list)

Succesfull insert


In [399]:
person_list =[
{'sista_4': 5426, 'f_namn': 'Oscar', 'e_namn': 'Hedlund', 'f_dag': '2005-07-14', 'kon': 'M', 'nat': 'SWE', 'adress_id': 26, 'inflytt_datum': '2016-09-20'},
{'sista_4': 9142, 'f_namn': 'Christian', 'e_namn': 'Arvidsson', 'f_dag': '1953-08-16', 'kon': 'M', 'nat': 'SWE', 'adress_id': 3, 'inflytt_datum': '2009-05-08'},
{'sista_4': 2370, 'f_namn': 'Tommy', 'e_namn': 'Martinsson', 'f_dag': '1959-08-20', 'kon': 'M', 'nat': 'SWE', 'adress_id': 55, 'inflytt_datum': '2020-12-02'},
{'sista_4': 4195, 'f_namn': 'Kristina', 'e_namn': 'Wallin', 'f_dag': '1979-07-02', 'kon': 'K', 'nat': 'SWE', 'adress_id': 30, 'inflytt_datum': '1999-01-21'},
{'sista_4': 5455, 'f_namn': 'Katarina', 'e_namn': 'Jansson', 'f_dag': '1982-07-31', 'kon': 'K', 'nat': 'SWE', 'adress_id': 56, 'inflytt_datum': '2020-01-19'},
{'sista_4': 6172, 'f_namn': 'Carl', 'e_namn': 'Forsberg', 'f_dag': '1970-12-18', 'kon': 'M', 'nat': 'SWE', 'adress_id': 25, 'inflytt_datum': '1995-12-29'},
{'sista_4': 9760, 'f_namn': 'Robert', 'e_namn': 'Ali', 'f_dag': '1979-09-23', 'kon': 'M', 'nat': 'SWE', 'adress_id': 16, 'inflytt_datum': '2013-02-25'},
{'sista_4': 5932, 'f_namn': 'Monica', 'e_namn': 'Eklund', 'f_dag': '1964-01-30', 'kon': 'K', 'nat': 'SWE', 'adress_id': 38, 'inflytt_datum': '1984-08-05'},
{'sista_4': 2412, 'f_namn': 'Charlotte', 'e_namn': 'Lindholm', 'f_dag': '1933-04-08', 'kon': 'K', 'nat': 'NOR', 'adress_id': 30, 'inflytt_datum': '2004-04-13'},
{'sista_4': 5031, 'f_namn': 'Nils', 'e_namn': 'Eklund', 'f_dag': '1989-08-06', 'kon': 'M', 'nat': 'SWE', 'adress_id': 58, 'inflytt_datum': '2016-01-19'},
{'sista_4': 7447, 'f_namn': 'Leif', 'e_namn': 'Svensson', 'f_dag': '2018-02-27', 'kon': 'M', 'nat': 'SWE', 'adress_id': 49, 'inflytt_datum': '2021-08-25'},
{'sista_4': 8431, 'f_namn': 'Alice', 'e_namn': 'Persson', 'f_dag': '1973-10-24', 'kon': 'K', 'nat': 'SWE', 'adress_id': 10, 'inflytt_datum': '1999-04-29'},
{'sista_4': 3886, 'f_namn': 'Britt', 'e_namn': 'Mattsson', 'f_dag': '1967-03-04', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '1983-07-28'},
{'sista_4': 1411, 'f_namn': 'Annika', 'e_namn': 'Ali', 'f_dag': '1978-03-17', 'kon': 'K', 'nat': 'SWE', 'adress_id': 17, 'inflytt_datum': '1984-08-28'},
{'sista_4': 1609, 'f_namn': 'Linda', 'e_namn': 'Lundgren', 'f_dag': '1936-08-22', 'kon': 'K', 'nat': 'SWE', 'adress_id': 14, 'inflytt_datum': '2004-09-06'},
{'sista_4': 5753, 'f_namn': 'Daniel', 'e_namn': 'Persson', 'f_dag': '2012-09-25', 'kon': 'M', 'nat': 'SWE', 'adress_id': 56, 'inflytt_datum': '2021-11-15'},
{'sista_4': 1762, 'f_namn': 'Christer', 'e_namn': 'Berg', 'f_dag': '1970-07-14', 'kon': 'M', 'nat': 'SWE', 'adress_id': 35, 'inflytt_datum': '1983-12-19'},
{'sista_4': 4858, 'f_namn': 'Eric', 'e_namn': 'Arvidsson', 'f_dag': '2019-12-26', 'kon': 'M', 'nat': 'SWE', 'adress_id': 7, 'inflytt_datum': '2021-07-23'},
{'sista_4': 8275, 'f_namn': 'Viktor', 'e_namn': 'Hermansson', 'f_dag': '2017-05-06', 'kon': 'M', 'nat': 'SWE', 'adress_id': 39, 'inflytt_datum': '2021-09-19'},
{'sista_4': 3403, 'f_namn': 'Lennart', 'e_namn': 'Danielsson', 'f_dag': '1934-04-20', 'kon': 'M', 'nat': 'SWE', 'adress_id': 22, 'inflytt_datum': '1977-02-15'},
{'sista_4': 5204, 'f_namn': 'Elisabet', 'e_namn': 'Falk', 'f_dag': '1987-01-02', 'kon': 'K', 'nat': 'SWE', 'adress_id': 6, 'inflytt_datum': '2009-12-04'},
{'sista_4': 7640, 'f_namn': 'Irene', 'e_namn': 'Hassan', 'f_dag': '1938-01-31', 'kon': 'K', 'nat': 'SWE', 'adress_id': 3, 'inflytt_datum': '1996-06-22'},
{'sista_4': 7681, 'f_namn': 'Lena', 'e_namn': 'Nordin', 'f_dag': '2021-06-27', 'kon': 'K', 'nat': 'SWE', 'adress_id': 5, 'inflytt_datum': '2021-11-20'},
{'sista_4': 5722, 'f_namn': 'Malin', 'e_namn': 'Danielsson', 'f_dag': '2005-08-09', 'kon': 'K', 'nat': 'GER', 'adress_id': 35, 'inflytt_datum': '2016-01-25'},
{'sista_4': 3326, 'f_namn': 'Oscar', 'e_namn': 'Johnsson', 'f_dag': '1965-10-17', 'kon': 'M', 'nat': 'SWE', 'adress_id': 45, 'inflytt_datum': '2009-01-09'},
{'sista_4': 4607, 'f_namn': 'Peter', 'e_namn': 'Hassan', 'f_dag': '1937-12-17', 'kon': 'M', 'nat': 'SWE', 'adress_id': 31, 'inflytt_datum': '1967-05-22'},
{'sista_4': 9866, 'f_namn': 'Mattias', 'e_namn': 'Axelsson', 'f_dag': '1941-07-23', 'kon': 'M', 'nat': 'SWE', 'adress_id': 27, 'inflytt_datum': '1972-07-02'},
{'sista_4': 7484, 'f_namn': 'Maria', 'e_namn': 'Lundberg', 'f_dag': '1995-11-25', 'kon': 'K', 'nat': 'FIN', 'adress_id': 13, 'inflytt_datum': '2007-01-04'},
{'sista_4': 7719, 'f_namn': 'Lisa', 'e_namn': 'Lindqvist', 'f_dag': '1978-05-31', 'kon': 'K', 'nat': 'SWE', 'adress_id': 45, 'inflytt_datum': '2020-10-21'},
{'sista_4': 2449, 'f_namn': 'Andreas', 'e_namn': 'Arvidsson', 'f_dag': '1997-07-13', 'kon': 'M', 'nat': 'SWE', 'adress_id': 39, 'inflytt_datum': '2017-06-30'},
{'sista_4': 7086, 'f_namn': 'Gustav', 'e_namn': 'Nyberg', 'f_dag': '1940-02-24', 'kon': 'M', 'nat': 'SWE', 'adress_id': 21, 'inflytt_datum': '1959-01-22'},
{'sista_4': 9090, 'f_namn': 'Tomas', 'e_namn': 'Johansson', 'f_dag': '1994-06-21', 'kon': 'M', 'nat': 'SWE', 'adress_id': 49, 'inflytt_datum': '2012-03-31'},
{'sista_4': 7959, 'f_namn': 'Maria', 'e_namn': 'Henriksson', 'f_dag': '2016-12-12', 'kon': 'K', 'nat': 'SWE', 'adress_id': 51, 'inflytt_datum': '2018-03-18'},
{'sista_4': 3394, 'f_namn': 'Leif', 'e_namn': 'Fredriksson', 'f_dag': '2011-05-28', 'kon': 'M', 'nat': 'SWE', 'adress_id': 42, 'inflytt_datum': '2021-04-24'},
{'sista_4': 2518, 'f_namn': 'Therese', 'e_namn': 'Berglund', 'f_dag': '2018-04-29', 'kon': 'K', 'nat': 'SWE', 'adress_id': 46, 'inflytt_datum': '2021-06-14'},
{'sista_4': 1936, 'f_namn': 'Oskar', 'e_namn': 'Falk', 'f_dag': '1969-11-13', 'kon': 'M', 'nat': 'SWE', 'adress_id': 47, 'inflytt_datum': '1993-09-19'},
{'sista_4': 9591, 'f_namn': 'Roger', 'e_namn': 'Lundin', 'f_dag': '2003-11-11', 'kon': 'M', 'nat': 'SWE', 'adress_id': 54, 'inflytt_datum': '2007-06-17'},
{'sista_4': 8457, 'f_namn': 'Charlotte', 'e_namn': 'Lundqvist', 'f_dag': '2003-08-24', 'kon': 'K', 'nat': 'SWE', 'adress_id': 58, 'inflytt_datum': '2018-08-27'},
{'sista_4': 1630, 'f_namn': 'John', 'e_namn': 'Blom', 'f_dag': '1986-10-09', 'kon': 'M', 'nat': 'SWE', 'adress_id': 10, 'inflytt_datum': '2010-05-22'},
{'sista_4': 8917, 'f_namn': 'Gustav', 'e_namn': 'Isaksson', 'f_dag': '1935-11-08', 'kon': 'M', 'nat': 'SWE', 'adress_id': 19, 'inflytt_datum': '1968-10-13'},
{'sista_4': 6873, 'f_namn': 'Amanda', 'e_namn': 'Hansson', 'f_dag': '1946-05-22', 'kon': 'K', 'nat': 'GER', 'adress_id': 46, 'inflytt_datum': '2011-09-01'},
{'sista_4': 3005, 'f_namn': 'Anders', 'e_namn': 'Nordin', 'f_dag': '2009-04-03', 'kon': 'M', 'nat': 'SWE', 'adress_id': 22, 'inflytt_datum': '2014-07-04'},
{'sista_4': 7618, 'f_namn': 'Christina', 'e_namn': 'Martinsson', 'f_dag': '1938-10-13', 'kon': 'K', 'nat': 'SWE', 'adress_id': 23, 'inflytt_datum': '1959-01-19'},
{'sista_4': 7006, 'f_namn': 'Erik', 'e_namn': 'Norberg', 'f_dag': '2000-01-30', 'kon': 'M', 'nat': 'SWE', 'adress_id': 6, 'inflytt_datum': '2015-12-01'},
{'sista_4': 6155, 'f_namn': 'Carina', 'e_namn': 'Gustavsson', 'f_dag': '1954-02-02', 'kon': 'K', 'nat': 'SWE', 'adress_id': 54, 'inflytt_datum': '2004-04-13'},
{'sista_4': 2741, 'f_namn': 'Johanna', 'e_namn': 'Dahlberg', 'f_dag': '1947-09-03', 'kon': 'K', 'nat': 'SWE', 'adress_id': 39, 'inflytt_datum': '1977-05-02'},
{'sista_4': 5169, 'f_namn': 'Elisabet', 'e_namn': 'Sundberg', 'f_dag': '1934-06-07', 'kon': 'K', 'nat': 'SWE', 'adress_id': 1, 'inflytt_datum': '1952-05-26'},
{'sista_4': 9543, 'f_namn': 'Emil', 'e_namn': 'Nilsson', 'f_dag': '1937-04-02', 'kon': 'M', 'nat': 'SWE', 'adress_id': 14, 'inflytt_datum': '1978-01-09'},
{'sista_4': 4946, 'f_namn': 'Ulla', 'e_namn': 'Lundin', 'f_dag': '1983-10-02', 'kon': 'K', 'nat': 'SWE', 'adress_id': 16, 'inflytt_datum': '1984-05-22'},
{'sista_4': 3475, 'f_namn': 'Marie', 'e_namn': 'Lind', 'f_dag': '2006-11-17', 'kon': 'K', 'nat': 'SWE', 'adress_id': 16, 'inflytt_datum': '2008-11-27'},
{'sista_4': 2887, 'f_namn': 'Birgitta', 'e_namn': 'Gunnarsson', 'f_dag': '1959-08-22', 'kon': 'K', 'nat': 'SWE', 'adress_id': 3, 'inflytt_datum': '1990-10-24'},
{'sista_4': 6528, 'f_namn': 'Per', 'e_namn': 'Carlsson', 'f_dag': '2013-06-01', 'kon': 'M', 'nat': 'SWE', 'adress_id': 7, 'inflytt_datum': '2021-06-17'},
{'sista_4': 4676, 'f_namn': 'Andreas', 'e_namn': 'Axelsson', 'f_dag': '1977-03-18', 'kon': 'M', 'nat': 'SWE', 'adress_id': 1, 'inflytt_datum': '1999-08-13'},
{'sista_4': 1351, 'f_namn': 'Elisabeth', 'e_namn': 'Sundberg', 'f_dag': '1938-11-14', 'kon': 'K', 'nat': 'SWE', 'adress_id': 21, 'inflytt_datum': '1954-09-11'},
{'sista_4': 1075, 'f_namn': 'Robert', 'e_namn': 'Karlsson', 'f_dag': '1983-07-31', 'kon': 'M', 'nat': 'SWE', 'adress_id': 33, 'inflytt_datum': '2014-06-26'},
{'sista_4': 9358, 'f_namn': 'Louise', 'e_namn': 'Hassan', 'f_dag': '1981-03-19', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '2014-01-16'},
{'sista_4': 4278, 'f_namn': 'Anton', 'e_namn': 'Samuelsson', 'f_dag': '1954-08-20', 'kon': 'M', 'nat': 'SIR', 'adress_id': 55, 'inflytt_datum': '2007-10-31'},
{'sista_4': 8772, 'f_namn': 'Ingvar', 'e_namn': 'Hermansson', 'f_dag': '1957-08-25', 'kon': 'M', 'nat': 'SWE', 'adress_id': 8, 'inflytt_datum': '1968-02-08'},
{'sista_4': 6099, 'f_namn': 'Gunnar', 'e_namn': 'Berggren', 'f_dag': '1965-12-12', 'kon': 'M', 'nat': 'SWE', 'adress_id': 26, 'inflytt_datum': '1972-05-10'},
{'sista_4': 4136, 'f_namn': 'Elin', 'e_namn': 'Henriksson', 'f_dag': '2010-11-01', 'kon': 'K', 'nat': 'SWE', 'adress_id': 40, 'inflytt_datum': '2014-11-18'},
{'sista_4': 6206, 'f_namn': 'Ida', 'e_namn': 'Blomqvist', 'f_dag': '1959-07-20', 'kon': 'K', 'nat': 'SWE', 'adress_id': 9, 'inflytt_datum': '1989-02-24'},
{'sista_4': 6812, 'f_namn': 'Ulla', 'e_namn': 'Bengtsson', 'f_dag': '1995-02-12', 'kon': 'K', 'nat': 'GER', 'adress_id': 49, 'inflytt_datum': '2011-06-29'},
{'sista_4': 8178, 'f_namn': 'Sofie', 'e_namn': 'Mattsson', 'f_dag': '2006-02-18', 'kon': 'K', 'nat': 'SWE', 'adress_id': 57, 'inflytt_datum': '2018-07-02'},
{'sista_4': 7175, 'f_namn': 'Eric', 'e_namn': 'Lind', 'f_dag': '1981-03-29', 'kon': 'M', 'nat': 'SWE', 'adress_id': 31, 'inflytt_datum': '1992-11-01'},
{'sista_4': 9852, 'f_namn': 'Robert', 'e_namn': 'Abrahamsson', 'f_dag': '1954-11-19', 'kon': 'M', 'nat': 'SWE', 'adress_id': 54, 'inflytt_datum': '1973-02-12'},
{'sista_4': 4682, 'f_namn': 'Gunnar', 'e_namn': 'Wallin', 'f_dag': '1942-05-22', 'kon': 'M', 'nat': 'SWE', 'adress_id': 55, 'inflytt_datum': '1995-07-05'},
{'sista_4': 3455, 'f_namn': 'Tommy', 'e_namn': 'Fransson', 'f_dag': '1954-07-03', 'kon': 'M', 'nat': 'SWE', 'adress_id': 54, 'inflytt_datum': '2006-10-30'},
{'sista_4': 6326, 'f_namn': 'Viktor', 'e_namn': 'Henriksson', 'f_dag': '2013-06-25', 'kon': 'M', 'nat': 'FIN', 'adress_id': 18, 'inflytt_datum': '2020-01-15'},
{'sista_4': 7798, 'f_namn': 'Patrik', 'e_namn': 'Nilsson', 'f_dag': '1982-04-25', 'kon': 'M', 'nat': 'SWE', 'adress_id': 11, 'inflytt_datum': '2014-12-27'},
{'sista_4': 3847, 'f_namn': 'Viktoria', 'e_namn': 'Norberg', 'f_dag': '1991-08-06', 'kon': 'K', 'nat': 'SWE', 'adress_id': 17, 'inflytt_datum': '1996-08-26'},
{'sista_4': 1828, 'f_namn': 'Kjell', 'e_namn': 'Henriksson', 'f_dag': '2017-04-09', 'kon': 'M', 'nat': 'SWE', 'adress_id': 6, 'inflytt_datum': '2019-02-26'},
{'sista_4': 4721, 'f_namn': 'Daniel', 'e_namn': 'Bergman', 'f_dag': '1974-11-11', 'kon': 'M', 'nat': 'SIR', 'adress_id': 52, 'inflytt_datum': '1983-05-06'},
{'sista_4': 5214, 'f_namn': 'Mats', 'e_namn': 'Pettersson', 'f_dag': '1990-10-09', 'kon': 'M', 'nat': 'DEN', 'adress_id': 17, 'inflytt_datum': '2012-12-10'},
{'sista_4': 3563, 'f_namn': 'Leif', 'e_namn': 'Samuelsson', 'f_dag': '2013-12-09', 'kon': 'M', 'nat': 'SWE', 'adress_id': 44, 'inflytt_datum': '2015-07-28'},
{'sista_4': 1860, 'f_namn': 'Roland', 'e_namn': 'Dahlberg', 'f_dag': '1976-02-07', 'kon': 'M', 'nat': 'SWE', 'adress_id': 17, 'inflytt_datum': '2003-02-09'},
{'sista_4': 6690, 'f_namn': 'Viola', 'e_namn': 'Samuelsson', 'f_dag': '1935-03-26', 'kon': 'K', 'nat': 'SWE', 'adress_id': 43, 'inflytt_datum': '1938-06-12'},
{'sista_4': 5864, 'f_namn': 'Helena', 'e_namn': 'Nordin', 'f_dag': '1990-10-31', 'kon': 'K', 'nat': 'SWE', 'adress_id': 12, 'inflytt_datum': '1998-06-09'},
{'sista_4': 7850, 'f_namn': 'Johanna', 'e_namn': 'Gunnarsson', 'f_dag': '1991-11-24', 'kon': 'K', 'nat': 'SWE', 'adress_id': 37, 'inflytt_datum': '2003-01-01'},
{'sista_4': 8721, 'f_namn': 'Astrid', 'e_namn': 'Axelsson', 'f_dag': '1934-05-20', 'kon': 'K', 'nat': 'SWE', 'adress_id': 38, 'inflytt_datum': '2017-12-03'},
{'sista_4': 4603, 'f_namn': 'Carl', 'e_namn': 'Lindqvist', 'f_dag': '1938-10-20', 'kon': 'M', 'nat': 'FIN', 'adress_id': 47, 'inflytt_datum': '1962-09-10'},
{'sista_4': 4056, 'f_namn': 'Therese', 'e_namn': 'Berggren', 'f_dag': '1996-03-24', 'kon': 'K', 'nat': 'SWE', 'adress_id': 7, 'inflytt_datum': '2005-12-13'},
{'sista_4': 2848, 'f_namn': 'Margareta', 'e_namn': 'Blom', 'f_dag': '1935-10-27', 'kon': 'K', 'nat': 'SWE', 'adress_id': 30, 'inflytt_datum': '1941-07-18'},
{'sista_4': 4171, 'f_namn': 'Ingvar', 'e_namn': 'Svensson', 'f_dag': '2002-04-23', 'kon': 'M', 'nat': 'SWE', 'adress_id': 35, 'inflytt_datum': '2013-07-09'},
{'sista_4': 5006, 'f_namn': 'Gunilla', 'e_namn': 'Arvidsson', 'f_dag': '1960-05-05', 'kon': 'K', 'nat': 'SWE', 'adress_id': 41, 'inflytt_datum': '1973-05-22'},
{'sista_4': 6078, 'f_namn': 'Caroline', 'e_namn': 'Carlsson', 'f_dag': '2017-02-07', 'kon': 'K', 'nat': 'SWE', 'adress_id': 4, 'inflytt_datum': '2019-04-28'},
{'sista_4': 3494, 'f_namn': 'John', 'e_namn': 'Gunnarsson', 'f_dag': '1954-03-20', 'kon': 'M', 'nat': 'SWE', 'adress_id': 51, 'inflytt_datum': '1994-07-09'},
{'sista_4': 4450, 'f_namn': 'Gustaf', 'e_namn': 'Holmgren', 'f_dag': '2018-01-09', 'kon': 'M', 'nat': 'SWE', 'adress_id': 25, 'inflytt_datum': '2020-11-29'},
{'sista_4': 4704, 'f_namn': 'Ingemar', 'e_namn': 'Axelsson', 'f_dag': '1956-04-02', 'kon': 'M', 'nat': 'SWE', 'adress_id': 37, 'inflytt_datum': '1995-12-24'},
{'sista_4': 8126, 'f_namn': 'Tommy', 'e_namn': 'Ahmed', 'f_dag': '1949-06-01', 'kon': 'M', 'nat': 'SWE', 'adress_id': 21, 'inflytt_datum': '1993-11-22'},
{'sista_4': 5060, 'f_namn': 'Mikael', 'e_namn': 'Berggren', 'f_dag': '1961-09-06', 'kon': 'M', 'nat': 'SWE', 'adress_id': 57, 'inflytt_datum': '1974-11-23'},
{'sista_4': 4283, 'f_namn': 'Arne', 'e_namn': 'Ali', 'f_dag': '1970-05-01', 'kon': 'M', 'nat': 'SWE', 'adress_id': 50, 'inflytt_datum': '1999-11-28'},
{'sista_4': 2669, 'f_namn': 'Monica', 'e_namn': 'Johnsson', 'f_dag': '1973-11-12', 'kon': 'K', 'nat': 'GER', 'adress_id': 5, 'inflytt_datum': '2004-04-23'},
{'sista_4': 1535, 'f_namn': 'Anders', 'e_namn': 'Holmberg', 'f_dag': '1948-10-15', 'kon': 'M', 'nat': 'SWE', 'adress_id': 32, 'inflytt_datum': '1991-06-04'},
{'sista_4': 9765, 'f_namn': 'Axel', 'e_namn': 'Gustafsson', 'f_dag': '1934-08-18', 'kon': 'M', 'nat': 'SWE', 'adress_id': 48, 'inflytt_datum': '1964-11-09'},
{'sista_4': 4468, 'f_namn': 'Sandra', 'e_namn': 'Abrahamsson', 'f_dag': '1937-02-14', 'kon': 'K', 'nat': 'SWE', 'adress_id': 52, 'inflytt_datum': '1943-06-28'},
{'sista_4': 9674, 'f_namn': 'Charlotte', 'e_namn': 'Lundberg', 'f_dag': '2017-03-17', 'kon': 'K', 'nat': 'SWE', 'adress_id': 29, 'inflytt_datum': '2020-07-31'},
{'sista_4': 4699, 'f_namn': 'Nils', 'e_namn': 'Ek', 'f_dag': '1968-10-05', 'kon': 'M', 'nat': 'SWE', 'adress_id': 34, 'inflytt_datum': '2019-01-21'},
{'sista_4': 6560, 'f_namn': 'Simon', 'e_namn': 'Dahlberg', 'f_dag': '1963-12-17', 'kon': 'M', 'nat': 'SWE', 'adress_id': 3, 'inflytt_datum': '1977-08-09'},
{'sista_4': 6119, 'f_namn': 'Marianne', 'e_namn': 'Nordin', 'f_dag': '1970-10-15', 'kon': 'K', 'nat': 'SWE', 'adress_id': 6, 'inflytt_datum': '2005-08-12'},
{'sista_4': 1970, 'f_namn': 'Sven', 'e_namn': 'Hassan', 'f_dag': '1970-08-26', 'kon': 'M', 'nat': 'FIN', 'adress_id': 31, 'inflytt_datum': '1977-02-19'}
]

In [400]:
insert_data(personer, person_list)

Succesfull insert


In [428]:
# fdag future
to_ins = [{'sista_4': 1111, 'f_namn': 'Emma', 'e_namn': 'Andersson', 'f_dag': '2023-01-01', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '2023-01-01'}]
# incorrect date
to_ins2 = [{'sista_4': 2222, 'f_namn': 'Emma', 'e_namn': 'Sjögren', 'f_dag': '2022-11-31', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '2023-01-01'}]
# too old
# to_ins3 =[{'personnummer': 197008261970, 'f_namn': 'Sven', 'e_namn': 'Hassan', 'f_dag': '1970-08-26', 'kon': 'M', 'nat': 'FIN', 'adress_id': 31, 'inflytt_datum': '1977-02-19'}]
to_ins3 = [{'sista_4': 2220, 'f_namn': 'EmmaOld', 'e_namn': 'Sjögren', 'f_dag': '1910-10-01', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '2022-01-01'}]
# future flytt
to_ins4 = [{'sista_4': 1111, 'f_namn': 'Emma', 'e_namn': 'Andersson', 'f_dag': '2022-01-01', 'kon': 'K', 'nat': 'SWE', 'adress_id': 20, 'inflytt_datum': '2023-01-01'}]
# valid
to_ins5 = [{'sista_4': 1111, 'f_namn': 'Sven', 'e_namn': 'Hassan', 'f_dag': '1970-08-26', 'kon': 'M', 'nat': 'FIN', 'adress_id': 31, 'inflytt_datum': '1977-02-19'}]
# insert_data(personer, to_ins)
# insert_data(personer, to_ins2)
# insert_data(personer, to_ins3)
# insert_data(personer, to_ins4)
insert_data(personer, to_ins5)


Succesfull insert


In [424]:

read_data(target_col='*', table_in=personer)

,sista_4,f_namn,e_namn,f_dag,kon,nat,adress_id,inflytt_datum,registrerings_datum
0,1075,Robert,Karlsson,1983-07-31,M,SWE,33,2014-06-26,2022-11-28
1,1111,Sven,Hassan,1970-08-26,M,FIN,31,1977-02-19,2022-11-28
2,1351,Elisabeth,Sundberg,1938-11-14,K,SWE,21,1954-09-11,2022-11-28
3,1411,Annika,Ali,1978-03-17,K,SWE,17,1984-08-28,2022-11-28
4,1535,Anders,Holmberg,1948-10-15,M,SWE,32,1991-06-04,2022-11-28
...,...,...,...,...,...,...,...,...,...
97,9674,Charlotte,Lundberg,2017-03-17,K,SWE,29,2020-07-31,2022-11-28
98,9760,Robert,Ali,1979-09-23,M,SWE,16,2013-02-25,2022-11-28
99,9765,Axel,Gustafsson,1934-08-18,M,SWE,48,1964-11-09,2022-11-28
100,9852,Robert,Abrahamsson,1954-11-19,M,SWE,54,1973-02-12,2022-11-28


### Update data function

**NOTE:**
__I assume that the updating of values is based on the persons personnummer, which in my case is a combination of the sista_4 and f_dag columns.__
- <u>table</u> argument is the table object
- <u>where_col</u> is the specific tables column object (ex.: users.c.sista_4), and specifies the lookup column (as in the WHERE command in SQL UPDATE)
- <u>where_col2</u> is the specific tables column object (ex.: users.c.f_dag), and specifies the lookup column (as in the WHERE command in SQL UPDATE)
- <u>to_col</u> is the specific tables column name in string format (ex.: 'nat'), and specifies the column where the update should take place
- <u>value</u> is a list of dictionaries with 2 items: 
    - **"where": lookup value pair**. This is the lookup value corresponding to the where_col argument, identifiying the row to update.
    - **"where2": lookup value pair**. This is the lookup value corresponding to the where_col2 argument, identifiying the row to update.
    - **"set": value to update to pair**. This is the update value corresponding to the to_col argument and will update the value found there.

In [431]:
def update_data(table, where_col, where_col2, to_col, value):
    stmt = (
    update(table)
    .where(where_col == bindparam("where")).where(where_col2 == bindparam("where2"))
    .values(**{to_col:bindparam("set")})
    )
    try:
        with engine.begin() as conn:
            result = conn.execute(stmt, value)
        print('Affected number of rows:' + str(result.rowcount))
        print(stmt)
    except Exception as e: print('Error:', e)

In [434]:
upd_vals = [
    {"where": 6560, "where2": '1963-12-17', "set": "ABC"},
    {"where": 6119, "where2": '1970-10-15', "set": "ABC"},
    {"where": 1970, "where2": '1970-08-26', "set": "ABC"}
]

In [435]:
update_data(personer, personer.c.sista_4, personer.c.f_dag, 'nat', upd_vals)

Affected number of rows:3
UPDATE person SET nat=:set WHERE person.sista_4 = :where AND person.f_dag = :where2


In [365]:
to_upd = [{'where':202301011111, 'set':'Sjögren'}]
update_data(personer, personer.c.personnummer, 'e_namn', to_upd)

Affected number of rows:1
UPDATE person SET e_namn=:set WHERE person.personnummer = :where


### Delete data function

**NOTE:**
__I assume that the deleting of values is based on the persons personnummer, which in my case is a combination of the sista_4 and f_dag columns.__
- <u>table</u> argument is a table object
- <u>where_col</u> is the specific tables column object (ex.: users.c.sista_4), and specifies the lookup column (as in the WHERE command in SQL DELETE)
- <u>where_col2</u> is the specific tables column object (ex.: users.c.f_dag), and specifies the lookup column (as in the WHERE command in SQL DELETE)
- <u>value</u> is the lookup value for where_col,  and identifies the row to delete

In [429]:
def delete_data(table, where_col, value, where_col2, value2):
    stmt = delete(table).where(where_col == value).where(where_col2 == value2)
    try:
        with engine.begin() as conn:
            result = conn.execute(stmt)
        print('Affected number of rows:' + str(result.rowcount))
        print(stmt)
    except Exception as e: print('Error:', e)

In [430]:
delete_data(personer, personer.c.sista_4, 1111, personer.c.f_dag, '1970-08-26')

Affected number of rows:1
DELETE FROM person WHERE person.sista_4 = :sista_4_1 AND person.f_dag = :f_dag_1


### Reading data

**NOTE:**
- <u>sql_in</u> can take complete SQL queries, and will executre them directly (optional)
- <u>table_in</u> is the target tables name (minimum this argument must be given)
- <u>target_col</u> takes a list of the columns to be returned, as default it is '*'. The list can contain aggregate functions, but in that case no non-aggregate functions can be provided (optional)
- <u>where can</u> take a string with a valid SQL WHERE expression (ex.: "kon = 'K'") (optional)

In [404]:
def read_data(sql_in=None, target_col='*', table_in=None, where=None):
    if sql_in:
        try:
            return pd.read_sql(sql_in, engine)
        except Exception as e: 
            print('Error:', e)
    elif where:
        sql = f"SELECT {', '.join(target_col)} FROM {table_in} WHERE {where}"
        return pd.read_sql(sql, engine)
        print(sql)
    else:
        sql = f"SELECT {', '.join(target_col)} FROM {table_in}"
        return pd.read_sql(sql, engine)

In [405]:
read_data(table_in = personer, target_col='*', where = "kon = 'K'")

,sista_4,f_namn,e_namn,f_dag,kon,nat,adress_id,inflytt_datum,registrerings_datum
0,1351,Elisabeth,Sundberg,1938-11-14,K,SWE,21,1954-09-11,2022-11-28
1,1411,Annika,Ali,1978-03-17,K,SWE,17,1984-08-28,2022-11-28
2,1609,Linda,Lundgren,1936-08-22,K,SWE,14,2004-09-06,2022-11-28
3,2412,Charlotte,Lindholm,1933-04-08,K,NOR,30,2004-04-13,2022-11-28
4,2518,Therese,Berglund,2018-04-29,K,SWE,46,2021-06-14,2022-11-28
5,2669,Monica,Johnsson,1973-11-12,K,GER,5,2004-04-23,2022-11-28
6,2741,Johanna,Dahlberg,1947-09-03,K,SWE,39,1977-05-02,2022-11-28
7,2848,Margareta,Blom,1935-10-27,K,SWE,30,1941-07-18,2022-11-28
8,2887,Birgitta,Gunnarsson,1959-08-22,K,SWE,3,1990-10-24,2022-11-28
9,3475,Marie,Lind,2006-11-17,K,SWE,16,2008-11-27,2022-11-28


### Window functions

### Select the n oldest people in a city

- Limitations: n must be a list

In [409]:
def n_oldest(n):
    sql = f"""WITH ranking AS (SELECT sista_4, f_namn, e_namn, f_dag, ort, RANK() OVER(PARTITION BY ort ORDER BY f_dag) r 
    FROM person p JOIN adress a ON p.adress_id = a.id)
SELECT sista_4, f_namn, e_namn, f_dag, TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0) ålder, ort, r
FROM ranking WHERE r IN ({', '.join(list(map(str, n)))}) ORDER BY ort;"""
    return pd.read_sql(sql, engine).style.hide(axis='index')

In [410]:
n_oldest(n=[1,2,5])

sista_4,f_namn,e_namn,f_dag,ålder,ort,r
1535,Anders,Holmberg,1948-10-15,74.000000,Älvsjö,1
4136,Elin,Henriksson,2010-11-01,12.000000,Älvsjö,2
6119,Marianne,Nordin,1970-10-15,52.000000,Anderstorp,1
5204,Elisabet,Falk,1987-01-02,35.000000,Anderstorp,2
9674,Charlotte,Lundberg,2017-03-17,5.000000,Bromma,1
7640,Irene,Hassan,1938-01-31,84.000000,Falkenberg,1
9142,Christian,Arvidsson,1953-08-16,69.000000,Falkenberg,2
6172,Carl,Forsberg,1970-12-18,51.000000,Falkenberg,5
7618,Christina,Martinsson,1938-10-13,84.000000,Grimeton,1
6099,Gunnar,Berggren,1965-12-12,57.000000,Hägersten,1


### Average age per location

In [411]:
def avg_age_per_loc():
    sql = """SELECT DISTINCT ort, AVG(DATEDIFF(CURDATE(), f_dag)/365) OVER(PARTITION BY ort) AS avg_age
FROM person p JOIN adress a ON p.adress_id = a.id;"""
    return pd.read_sql(sql, engine).style.hide(axis='index')

In [412]:
avg_age_per_loc()

ort,avg_age
Älvsjö,43.126050
Anderstorp,29.142475
Bromma,5.704100
Falkenberg,48.844650
Grimeton,84.183600
Hägersten,37.193150
Halmstad,32.098600
Hargshamn,66.961650
Huddinge,40.108225
Johanneshov,68.620550


### Min, maxage per ort

In [413]:
def min_max_per_loc():
    sql="""WITH ages AS (SELECT sista_4, f_namn, e_namn, ort, TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0) ålder    
FROM person p JOIN adress a ON p.adress_id = a.id ORDER BY ålder), 
min_max AS (SELECT sista_4, f_namn, e_namn, ort, ålder,
    MIN(ålder) OVER(PARTITION BY ort ORDER BY ålder ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS min_age,
    MAX(ålder) OVER(PARTITION BY ort ORDER BY ålder ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS max_age
FROM ages ORDER BY ort)
SELECT sista_4, f_namn, e_namn, ort, ålder, CASE WHEN ålder = min_age THEN 'Yngsta' WHEN ålder = max_age THEN 'Äldsta'
END 'Yngsta eller Äldsta' 
FROM min_max WHERE ålder = min_age OR ålder = max_age;"""
    return pd.read_sql(sql, engine).style.hide(axis='index')

In [415]:
min_max_per_loc()

sista_4,f_namn,e_namn,ort,ålder,Yngsta eller Äldsta
4136,Elin,Henriksson,Älvsjö,12.000000,Yngsta
1535,Anders,Holmberg,Älvsjö,74.000000,Äldsta
1828,Kjell,Henriksson,Anderstorp,5.000000,Yngsta
6119,Marianne,Nordin,Anderstorp,52.000000,Äldsta
9674,Charlotte,Lundberg,Bromma,5.000000,Yngsta
4450,Gustaf,Holmgren,Falkenberg,4.000000,Yngsta
7640,Irene,Hassan,Falkenberg,84.000000,Äldsta
7618,Christina,Martinsson,Grimeton,84.000000,Yngsta
5426,Oscar,Hedlund,Hägersten,17.000000,Yngsta
6099,Gunnar,Berggren,Hägersten,57.000000,Äldsta


### Standard deviation of ages per location

**NOTE:**
- if the 'ordered' argument is set to True the query will be ordered by the standard deviation

In [418]:
def sd_per_loc(ordered=False):
    sql = """WITH ages AS (SELECT sista_4, f_dag, ort, TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0) ålder,
    AVG(TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0))  OVER(PARTITION BY ort) ort_avg,
    POWER(TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0) - AVG(TRUNCATE(DATEDIFF(CURDATE(), f_dag)/365, 0))  OVER(PARTITION BY ort),2) diff
FROM person p JOIN adress a ON p.adress_id = a.id GROUP BY sista_4, f_dag, ort ORDER BY ort)
SELECT ort, ROUND(SQRT(SUM(diff) / COUNT(DISTINCT sista_4, f_dag)), 2) sd, COUNT(DISTINCT sista_4, f_dag) 'num of habitants'
FROM ages GROUP BY ort"""
    if not ordered:
        return pd.read_sql(sql, engine).style.hide(axis='index')
    else:        
        sql += ' ORDER BY sd'
        return pd.read_sql(sql, engine).style.hide(axis='index')

In [419]:
sd_per_loc(ordered=True)

ort,sd,num of habitants
Sala,0.000000,1
Västra Frölunda,0.000000,1
Bromma,0.000000,1
Långås,0.000000,1
Grimeton,0.000000,1
Linköping,0.000000,1
Halmstad,0.000000,1
Vendelsö,0.000000,1
Oskarström,0.000000,1
Surahammar,0.000000,1


### People living at the same address since the longest time per ort

In [420]:
def longest_at_loc():
    sql = """WITH temp AS (SELECT sista_4, f_namn, e_namn, adress_id, ort, inflytt_datum, TRUNCATE(DATEDIFF(CURDATE(), inflytt_datum)/365.25, 1) year_lived_at_spot
FROM person p JOIN adress a ON a.id = p.adress_id ORDER BY ort), 
temp2 AS (SELECT sista_4, f_namn, e_namn, adress_id, ort, inflytt_datum, year_lived_at_spot, MAX(year_lived_at_spot) OVER(PARTITION BY ort) max_at_spot
FROM temp)
SELECT sista_4, f_namn, e_namn, adress_id, ort, inflytt_datum, year_lived_at_spot 'bodde på samma plats för'
FROM temp2 WHERE year_lived_at_spot = max_at_spot;"""
    return pd.read_sql(sql, engine).style.hide(axis='index')

In [421]:
longest_at_loc()

sista_4,f_namn,e_namn,adress_id,ort,inflytt_datum,bodde på samma plats för
1535,Anders,Holmberg,32,Älvsjö,1991-06-04,31.400000
6119,Marianne,Nordin,6,Anderstorp,2005-08-12,17.200000
9674,Charlotte,Lundberg,29,Bromma,2020-07-31,2.300000
6560,Simon,Dahlberg,3,Falkenberg,1977-08-09,45.300000
7618,Christina,Martinsson,23,Grimeton,1959-01-19,63.800000
6099,Gunnar,Berggren,26,Hägersten,1972-05-10,50.500000
5864,Helena,Nordin,12,Halmstad,1998-06-09,24.400000
4468,Sandra,Abrahamsson,52,Hargshamn,1943-06-28,79.400000
2741,Johanna,Dahlberg,39,Huddinge,1977-05-02,45.500000
4603,Carl,Lindqvist,47,Johanneshov,1962-09-10,60.200000
